In [1]:
import pandas as pd

In [73]:
dataset = pd.read_csv("/home/iftekhar/amiebot/Resources/amiebot_dataset/mobicontrol_ver14.csv")
dataset = dataset.rename(columns={"text": "Data", "page": "PageID"})
# dataset = extract_link_merge_title(dataset)
#dataset = corpus_merger(dataset)
dataset.head()

,PageID,file,url,title,Data
0,0,NaN,https://pol-japan.co.jp/products/help/v14/,MobiControl v14 Manual – MobiControl v14 Manual,カスタマイザーからこのタイトルを設定できます。
1,0,NaN,https://pol-japan.co.jp/products/help/v14/,MobiControl v14 Manual – MobiControl v14 Manual,カスタマイザーからこのサブタイトルを設定できます。
2,0,NaN,https://pol-japan.co.jp/products/help/v14/,MobiControl v14 Manual – MobiControl v14 Manual,MobiControl
3,0,NaN,https://pol-japan.co.jp/products/help/v14/,MobiControl v14 Manual – MobiControl v14 Manual,の設定順序
4,0,NaN,https://pol-japan.co.jp/products/help/v14/,MobiControl v14 Manual – MobiControl v14 Manual,MobiControlの設定順序


In [3]:
df = pd.read_csv('/home/iftekhar/AI-system/retrieval_Model/Page_Ranking_Experiment/pipelines/tokenized_cleaned_perpagedata.csv')
df.head()

,Unnamed: 0,PageID,URL,Data
0,0,0,https://pol-japan.co.jp/products/help/v14/,カスタマイザー この タイトル 設定 でき ます カスタマイザー この サブタイトル 設定 ...
1,1,1,https://pol-japan.co.jp/products/help/v14/ios_...,Apple 製品 MobiControl Manual Apple 製品 MobiCont...
2,2,2,https://pol-japan.co.jp/products/help/v14/plus...,Android 端末 MobiControl Manual Android 設定 モード...
3,3,3,https://pol-japan.co.jp/products/help/v14/wpc_...,Windows PC MobiControl Manual Windows PC 根本 ...
4,4,4,https://pol-japan.co.jp/products/help/v14/wm_o...,Windows Embedded MobiControl Manual Windows E...


In [4]:
import re
def single_character_remover(text):
    collector = []
    for items in text.split():
        if len(items) < 2:
            replaced = re.sub(r'[ぁ-んァ-ン]', '', items)
            replaced = re.sub(r'[A-Za-z]', '', replaced)
            replaced = re.sub(r'[0-9]', '', replaced)
            collector.append(replaced)
        else:
            collector.append(items)

    return ' '.join([temp.strip(' ') for temp in collector])

In [5]:
df['Data'] = df.Data.apply(lambda x: single_character_remover(x))

df.head()
df.to_csv('very_cleaned_dataset_mobicontrol.csv')

vocab = []
for index, col in df.iterrows():
    vocab.append(col['Data'].split())

vocabulary = list(set([v for sublist in vocab for v in sublist]))

In [24]:
len(vocabulary)

6242

In [42]:
from nltk.corpus import wordnet as wn

count = 0
collector = []
saver = []
for items in vocabulary:
    syms = [u.lemma_names('jpn') for u in list(wn.synsets(items, lang='jpn'))]
    if not syms:
        collector.append(items) 
    else:
        saver.append([items, [x for sublist in syms for x in sublist]])
    if count > 10:
        break
    count +=1

In [55]:
syms_dict = {}
for terms, syms in saver:
    syms_dict.update({terms : syms})

print(syms_dict)

{'進み': ['アドバンス', '前進', '前進運動', '行進', '進み', '進出', '進展', '進捗', '進歩', '進行', 'アドバンス', 'アドヴァンス', '上達', '前進', '向上', '改善', '改良', '発展', '発達', '進み', '進展', '進歩', '開展', '前進', '進み', '進出', '進歩', '進行'], '室': ['ハイム', 'アビタシオン', '住', '住い', '住まい', '住処', '住み処', '住宅', '住家', '住み家', '住居', '住屋', 'お宅', '宅', '実家', '室', '室家', '家', '家作', '家処', '家宅', '家居', '家屋', '家所', '宿り', '居', '居住', '居宅', '居館', '屋', '棲家', '棲み家', '自宅', '邸', '邸宅', '館', 'ルーム', '室', 'お座敷', '座敷', '御部屋', '房室', 'お部屋', '部屋', '間', 'セラー', '地下室', '室', '穴蔵', 'さや', '室', '鞘', 'フラウ', 'おかみさん', 'お上', 'お上さん', '上さん', '上様', '亭主もち', '亭主持', '亭主持ち', '人妻', '令夫人', '令室', '令閨', 'ご内儀', 'ご内室', '刀自', '嚊', '嚊左衛門', '夫人', '奥さん', '奥様', '女', '女君', '女性', '女房', '妻', '妻女', '妻室', '婦人', '嫁', '嫁さん', '嬢子', '嬶', '室', '室家', '家内', '家妻', '家桜', '御上', '御上さん', '御内', '御内儀', '御内室', '御っ母', '愚妻', 'ご新造', 'お方', 'おっ母', '細君', '裏方'], '端末': ['端末', '端末機', 'エンド', '末梢', '末端', '果て', '極端', '端', '端こ', '端っこ', '端末', '端部', '詰', '詰め', '先', '先っぽ', '先端', '外れ', '尖端', '尻', '尻っぽ', '尻っ方', '尾端', '後端', '最後', '末', '末方', 

In [ ]:
a_file = open("vocabulary_synonyms.json", "w")
json.dump(syms_dict, a_file)
a_file.close()

In [36]:
with open('/home/iftekhar/amiebot/exp_amiecore/amieCore/amie_core/core/retriever/Page_Ranking_Experiment'
          '/pipelines/vocabulary.txt') as f: 
    lines = f.read().splitlines()

In [37]:
lines

['全員',
 'CLASSES',
 '防ぎ',
 '付い',
 '同士',
 '変える',
 'Reporting',
 '危険',
 '気づく',
 'issuer',
 'フォアグラウンド',
 'こんにちは',
 '直前',
 'Google',
 '戻れ',
 'ストアアプリ',
 '新規',
 '拒否',
 'ボックス',
 'タイ',
 'わかる',
 'Markup',
 'HOST',
 '隠れ',
 'SSO',
 'トップクラス',
 '低下',
 '指令',
 '稀',
 'set',
 '示す',
 '潜在',
 'Assistive',
 'メーカー',
 'Watch',
 '壁紙',
 'Excluded',
 'May',
 '自社',
 'メンバー',
 'message',
 'surfs',
 'ミラー',
 'Macmini',
 '混載',
 'つい',
 'メタデータ',
 'jpg',
 '待ち受け',
 '形成',
 '見当たら',
 'ログアウト',
 '付託',
 'TS',
 '間',
 'メッセージ',
 'グランド',
 'pac',
 '収',
 'Denso',
 'Lockdown',
 'Core',
 'Cortana',
 '取り出し',
 'Deployment',
 'きい',
 'ランチャー',
 '社',
 'づける',
 'ケーブル',
 'Product',
 'グレード',
 'APNs',
 'レジストリパス',
 'CoreTelephony',
 '浮動',
 'it',
 'kioskdata',
 'Destination',
 '連想',
 'アピアランス',
 'せよ',
 'MDM',
 '分離',
 'running',
 '抜く',
 '門',
 'get',
 'リビジョン',
 '特権',
 '私製',
 'Extractor',
 'hanako',
 'XML',
 'Open',
 'サインアウト',
 '解凍',
 'TIMER',
 '予備',
 'apk',
 'スター',
 '溜まっ',
 'たとえ',
 '盗ん',
 '券',
 'New',
 '互換',
 'SEForAndroidStatus',
 'クローズ',
 'assign',

In [35]:
collector

['次回',
 'とともに',
 '拾っ',
 'iBeacon',
 'parameter',
 '良けれ',
 'signedCSR',
 'mastarTOC',
 'installer',
 'および',
 'version',
 'Geofence',
 'TokyoSales',
 'CHAP',
 'スクリーニング',
 '態様',
 'IDOK',
 '送受',
 'Map',
 '決まっ',
 'after',
 '映し',
 '受け渡し',
 'プラグインアプリ',
 'BitLocker',
 'DoCoMo',
 'バーコードアイコン',
 'Rule',
 'shellexecute',
 'サイレントインストレーション',
 '再送',
 'コンスタンツ',
 'Pre',
 'rem',
 'インタラクティブ',
 'Profile',
 '短く',
 'Net',
 '行なわ',
 'ubuntu',
 'Vlue',
 'セキュアモード',
 '落とし',
 '未',
 '私製',
 '不揮発',
 'stype',
 'toLock',
 'バッチ',
 'fotaclient',
 'エクスプローラ',
 'ジェイルブレーク',
 'つながら',
 'iPod',
 'アクティベーション',
 'VMware',
 '記し',
 'CSS',
 '速',
 'インスタント',
 '取替え',
 'SSP',
 'abortpkg',
 '従量',
 'anaged',
 'title',
 'CHANGE',
 'apache',
 'ワークス',
 'SDCARD',
 '一括',
 'キーガードカメラ',
 '録画',
 'AA',
 'コピペ',
 'FederationMetadata',
 '双方',
 'ストアアプリ',
 'Okta',
 'キー',
 'Routing',
 'FailOver',
 '対し',
 'オフライン',
 'maps',
 'たとえ',
 'mobileconfig',
 'Update',
 'ジェイル',
 '赤丸',
 '動い',
 'エージェントアップグレード',
 '上向き',
 '本件',
 'EXIT',
 'コンンソール',
 'そういう',
 'mnt',
 'bt'

In [78]:
def corpus_merger(corpus):
    total_pages = corpus.PageID.unique()
    data = PageID = url = title = summary = []
    for i in list(total_pages):
        page_data = corpus[corpus.PageID == i].Data.values
        data.append(' '.join(list(page_data)))
        PageID.append(i)
        u = corpus[corpus.PageID == i].url.values
        url.append(' '.join(list(set(u))))
#         t = corpus[corpus.PageID == i].title.values
#         title.append(' '.join(list(set(t))))
#         s = corpus[corpus.PageID == i].Summary.values
#         summary.append(' '.join(list(set(s))))

    df = pd.DataFrame(zip(PageID, data, url), columns=["PageID", "Data", "URL"])
    return df

In [79]:
merged = corpus_merger(dataset)
merged.head()

,PageID,Data,URL
0,カスタマイザーからこのタイトルを設定できます。 カスタマイザーからこのサブタイトルを設定でき...,カスタマイザーからこのタイトルを設定できます。 カスタマイザーからこのサブタイトルを設定でき...,カスタマイザーからこのタイトルを設定できます。 カスタマイザーからこのサブタイトルを設定でき...
1,0,0,0
2,https://pol-japan.co.jp/products/help/v14/,https://pol-japan.co.jp/products/help/v14/,https://pol-japan.co.jp/products/help/v14/
3,Apple製品 MobiControl v14 Manual Apple 製品を MobiC...,Apple製品 MobiControl v14 Manual Apple 製品を MobiC...,Apple製品 MobiControl v14 Manual Apple 製品を MobiC...
4,1,1,1


In [70]:
title

array(['MobiControl v14 Manual – MobiControl v14 Manual',
       'MobiControl v14 Manual – MobiControl v14 Manual',
       'MobiControl v14 Manual – MobiControl v14 Manual',
       'MobiControl v14 Manual – MobiControl v14 Manual',
       'MobiControl v14 Manual – MobiControl v14 Manual',
       'MobiControl v14 Manual – MobiControl v14 Manual',
       'MobiControl v14 Manual – MobiControl v14 Manual',
       'MobiControl v14 Manual – MobiControl v14 Manual',
       'MobiControl v14 Manual – MobiControl v14 Manual',
       'MobiControl v14 Manual – MobiControl v14 Manual',
       'MobiControl v14 Manual – MobiControl v14 Manual',
       'MobiControl v14 Manual – MobiControl v14 Manual',
       'MobiControl v14 Manual – MobiControl v14 Manual',
       'MobiControl v14 Manual – MobiControl v14 Manual',
       'MobiControl v14 Manual – MobiControl v14 Manual',
       'MobiControl v14 Manual – MobiControl v14 Manual',
       'MobiControl v14 Manual – MobiControl v14 Manual',
       'MobiCo

In [54]:
df.Summary.loc[71030]

'端末OS共通の設定項目 – MobiControl v14 Manual CommonSettings '

In [62]:
df = corpus_urlmerger(df)
df.head()

,PageID,Data,Summary,URL,Title
0,0,カスタマイザーからこのタイトルを設定できます。 カスタマイザーからこのサブタイトルを設定でき...,MobiControl v14 Manual – MobiControl v14 Manual,https://pol-japan.co.jp/products/help/v14/,MobiControl v14 Manual – MobiControl v14 Manual
1,1,Apple製品 MobiControl v14 Manual Apple 製品を MobiC...,Apple製品 – MobiControl v14 Manual ios overall,https://pol-japan.co.jp/products/help/v14/ios_...,Apple製品 – MobiControl v14 Manual
2,2,Android端末 MobiControl v14 Manual Android の設定モー...,Android端末 – MobiControl v14 Manual plus overall,https://pol-japan.co.jp/products/help/v14/plus...,Android端末 – MobiControl v14 Manual
3,3,Windows PC MobiControl v14 Manual Windows PC の...,Windows PC – MobiControl v14 Manual wpc overall,https://pol-japan.co.jp/products/help/v14/wpc_...,Windows PC – MobiControl v14 Manual
4,4,Windows Embedded MobiControl v14 Manual Window...,Windows Embedded – MobiControl v14 Manual wm o...,https://pol-japan.co.jp/products/help/v14/wm_o...,Windows Embedded – MobiControl v14 Manual


df["Summary"] = df[df.columns[1:3]].apply(lambda x: ' '.join(x.dropna().astype(str)), axis=1)

In [47]:
df.tail()

,PageID,URL,Title,Data
440,440,https://pol-japan.co.jp/products/help/v14/SDK_...,SDK for iOS – MobiControl v14 Manual,SDK for iOS MobiControl v14 Manual MobiControl...
441,441,https://pol-japan.co.jp/products/help/v14/PSOv...,パッケージ・スタジオ – MobiControl v14 Manual,パッケージ・スタジオ MobiControl v14 Manual MobiControl ...
442,442,https://pol-japan.co.jp/products/help/v14/Self...,セルフサービスポータル – MobiControl v14 Manual,セルフサービスポータル MobiControl v14 Manual セルフサービスポータル...
443,443,https://pol-japan.co.jp/products/help/v14/Secu...,Exchangeへの不法アクセス防止 – MobiControl v14 Manual,Exchangeへの不法アクセス防止 MobiControl v14 Manual MS E...
444,444,https://pol-japan.co.jp/products/help/v14/Comm...,端末OS共通の設定項目 – MobiControl v14 Manual,端末OS共通の設定項目 MobiControl v14 Manual 端末 OS 共通の設定...


In [15]:
df.Summary.loc[1]

'https://pol-japan.co.jp/products/help/v14/ios_overall/ Apple製品 – MobiControl v14 Manual'

In [30]:
Data_cols = df_per_page.iloc[:,4:-1]

In [31]:
df_per_page['Merged_text'] = Data_cols.apply(
    lambda x: ' '.join(x.dropna().astype(str)),
    axis=1
)

In [33]:
page_link = df_per_page.iloc[:,0:3]


In [ ]:
output = pd.concat((page_link,df_per_page[['Merged_text']]), axis=1)
output.head()


In [ ]:
out = output.rename({0: 'page', 1: 'link', 2: 'title'}, axis='columns')
out.head()

In [ ]:
without_nan['parsed_link'] = link_text

s = without_nan.title + " " +without_nan.parsed_link
without_nan['Summary'] = s

In [ ]:
without_nan.head()

In [55]:
dataset = without_nan.rename(columns={"Merged_text":"Article"})
dataset.to_csv("dataset.csv")

In [9]:
import pandas as pd
df = pd.read_csv("/home/ifte/amiebot_project/MyAI/AI-system/Helpers/dataset.csv", usecols=['Article', 'Summary'])
df.head()

,Article,Summary
0,NaN,
1,Help Version: 14.4 14.3 14.2 14.1 14.0 13 12 1...,banner
2,NaN,MobiControl Help toc
3,Find pages containing: Match case Match whole ...,Search search
4,SOTI MobiControl Help SOTI MobiControl is an e...,Welcome to SOTI MobiControl Help start


In [ ]:
def remove_special_characters(text):
    regex = r'[^a-zA-Z0-9\s]'
    text = re.sub(regex,'',text)
    text = re.sub("\d+","",text)
    return text

import re
# from nltk.stem import PorterStemmer 
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
# nltk.download('averaged_perceptron_tagger')
Stopwords = set(stopwords.words('english'))
wordlemmatizer = WordNetLemmatizer()
# ps = PorterStemmer() 

def preprocess_text(sentence):
    sentence = remove_special_characters(sentence)
    tokenized_words = word_tokenize(sentence)
    tokenized_words = [word for word in tokenized_words if word not in Stopwords]
    tokenized_words = [word for word in tokenized_words if len(word) > 1]
    tokenized_words = [word.lower() for word in tokenized_words]
    tokenized_words = [wordlemmatizer.lemmatize(w) for w in tokenized_words]
#     tokenized_words = [ps.stem(w) for w in tokenized_words]
    return " ".join(tokenized_words)

df.Summary = df.Summary.apply(lambda x: preprocess_text(str(x)))
df.Article = df.Article.apply(lambda x: preprocess_text(str(x)))
df.head()

df["Summary"][52]

In [14]:
def remove_duplicate(text):        
    return re.sub(r'\b(\w+)( \1\b)+', r'\1', text)

df.Summary = df.Summary.apply(lambda x: remove_duplicate(str(x)))
df.Article = df.Article.apply(lambda x: remove_duplicate(str(x)))

df["Summary"][52]

'managing user console user'

In [64]:
import pandas as pd
import sys
import MeCab
import re
import random
from pathlib import Path
from urllib.parse import urlparse

mecab = MeCab.Tagger('-Owakati')


def corpus_preprocessing(dataset):
    dataset = pd.read_csv(dataset)
    dataset = dataset.rename(columns={"text": "Data", "page": "PageID"})
    dataset = extract_link_merge_title(dataset)
    dataset = corpus_merger(dataset)
    print(dataset.head())
    exit()
    dataset.Data = dataset.Data.apply(lambda x: mecab_tokenization(x))
    dataset.Data = dataset.Data.apply(lambda x: cleaner(x))
    dataset.Data = dataset.Data.apply(lambda x: single_character_remover(x))
    dataset.Summary = dataset.Summary.apply(lambda x: mecab_tokenization(x))
    dataset.Summary = dataset.Summary.apply(lambda x: cleaner(x))
    dataset.Summary = dataset.Summary.apply(lambda x: single_character_remover(x))
    return dataset


def query_corpus_processing(corpus):
    cus_ques = pd.read_csv(corpus)
    cus_ques.Question = cus_ques.Question.apply(lambda x: mecab_tokenization(x))
    cus_ques.Question = cus_ques.Question.apply(lambda x: single_character_remover(x))
    cus_ques.Question = cus_ques.Question.apply(lambda x: cleaner(x))
    return cus_ques


def get_stop_word_ja():
    stop_word_file = Path("/home/iftekhar/AI-system/Helpers/stop_word_ja.txt")
    with open(stop_word_file, encoding='utf-8') as f:
        stop_word_list = f.read().splitlines()
    return stop_word_list


def mecab_tokenization(text):
    q = mecab.parse(text)
    q_parts = q.split()

    return ' '.join([word for word in q_parts if not word in get_stop_word_ja()])


def single_character_remover(text):
    collector = []
    for items in text.split():
        if len(items) < 2:
            replaced = re.sub(r'[ぁ-んァ-ン]', '', items)
            replaced = re.sub(r'[A-Za-z]', '', replaced)
            replaced = re.sub(r'[0-9]', '', replaced)
            collector.append(replaced)
        else:
            collector.append(items)

    return ' '.join([temp.strip(' ') for temp in collector])


def corpus_merger(corpus):
    total_pages = corpus.PageID.unique()
    data = PageID = url = title = summary = []
    for i in list(total_pages):
        page_data = corpus[corpus.PageID == i].Data.values
        data.append(' '.join(list(page_data)))
        PageID.append(i)
        u = corpus[corpus.PageID == i].url.values
        url.append(' '.join(list(set(u))))
        t = corpus[corpus.PageID == i].title.values
        title.append(' '.join(list(set(t))))
        s = corpus[corpus.PageID == i].Summary.values
        summary.append(' '.join(list(set(s))))

    df = pd.DataFrame(zip(PageID, data, summary, url, title), columns=["PageID", "Data", "Summary", "URL", "Title"])
    return df


def corpus_per_page(corpus):
    total_pages = corpus.PageID.unique()
    data = []
    PageID = []
    for i in list(total_pages):
        page_data = corpus[corpus.PageID == i].Data.values
        data.append(' '.join(list(page_data)))
        PageID.append(i)
    df = pd.DataFrame(zip(data, PageID), columns=["Data", "PageID"])
    return df


def corpus_split(corpus, sentence_length):
    labels = corpus.PageID.unique()
    lines = []
    all_ids = []
    for i in list(labels):
        text_list = corpus[corpus.PageID == i].Data.values
        split_text = fixed_length_sentence(' '.join(text_list), sentence_length)
        ids = [i] * len(split_text)
        lines += split_text
        all_ids += ids
    split_corpus = pd.DataFrame(zip(lines, all_ids), columns=["Data", "PageID"])
    return split_corpus


def cleaner(text):
    collector = []
    for items in text.split():
        cleaned = clean_text(items)
        cleaned = re.sub(r"\s+", '', cleaned)
        if cleaned is not '' or cleaned is not ' ':
            collector.append(clean_text(items))

    return ' '.join(collector)


def clean_text(text):
    replaced = text.replace("\\", "")
    replaced = replaced.replace("+", "")
    replaced = re.sub('_', '', replaced)
    replaced = re.sub('\W+', ' ', replaced)
    replaced = re.sub(r'￥', '', replaced)  # 【】の除去
    replaced = re.sub(r'．', '', replaced)  # ・ の除去
    replaced = re.sub(r'｣', '', replaced)  # （）の除去
    replaced = re.sub(r'｢', '', replaced)  # ［］の除去
    replaced = re.sub(r'～', '', replaced)  # メンションの除去
    replaced = re.sub(r'｜', '', replaced)  # URLの除去
    replaced = re.sub(r'＠', '', replaced)  # 全角空白の除去
    replaced = re.sub(r'？', '', replaced)  # 数字の除去
    replaced = re.sub(r'％', '', replaced)
    replaced = re.sub(r'＝', '', replaced)
    replaced = re.sub(r'！', '', replaced)
    replaced = re.sub(r'｝', '', replaced)
    replaced = re.sub(r'：', '', replaced)
    replaced = re.sub(r'－', '', replaced)
    replaced = re.sub(r'･', '', replaced)
    replaced = re.sub(r'ｔ', '', replaced)
    replaced = re.sub(r'ｋ', '', replaced)
    replaced = re.sub(r'ｄ', '', replaced)
    replaced = re.sub(r'\d+', '', replaced)

    return replaced


def fixed_length_sentence(contents, word_limit):
    contents_list = contents.split()
    end = len(contents_list)
    count = 0
    collector = []
    line = []
    for items in contents_list:
        if count < word_limit - 1 and end > 1:
            collector.append(items)
            count += 1
        else:
            collector.append(items)
            line.append(' '.join(collector))
            collector = []
            count = 0
        end -= 1
    return line


def page_text_split(page_text, word_limit):
    page_text = page_text.split()
    chunks = [' '.join(page_text[i:i + word_limit]) for i in range(0,
                                                                   len(page_text), word_limit)]
    return chunks


def query_token_remover(query):
    query_list = query.split()
    query_list = [items for items in query_list if items not in question_maker.get_questions_delimiter_ja()]
    query_text = ' '.join(query_list)
    return query_text


def deep_clean(matched_sequence):
    rank = []
    collector = []
    ids_list = []
    for ids, items in matched_sequence:
        for sentences in items:
            clean_sentence = []
            for items in ''.join(sentences).split():
                if len(items) < 2:
                    items = re.sub(r'[ぁ-ん]', '', items)
                    items = re.sub(r'[ア-ン]', '', items)
                    items = re.sub(r'[A-Za-z]', '', items)
                    clean_sentence.append(items)
                else:
                    clean_sentence.append(items)
            cleaned_text = [cleaned for cleaned in clean_sentence if cleaned not in ['']]
            collector.append([ids, ' '.join(cleaned_text)])
    return collector


def extract_link_merge_title(corpus):
    link_text = []
    for index, col in corpus.iterrows():
        disassembled = urlparse(col['url'])
        only_url = re.sub('/products/help/v14/', '', disassembled.path)
        only_url = re.sub('.html', '', only_url)
        only_url = re.sub('_', ' ', only_url)
        only_url = re.sub('/', ' ', only_url)
        link_text.append(only_url)

    corpus['parsed_link'] = link_text
    corpus['Summary'] = corpus.title + " " + corpus.parsed_link

    return corpus
